# Problem Description
There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.

The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the `machine_id` along with the average time as `processing_time`, which should be rounded to 3 decimal places.

Return the result table in any order.

The result format is in the following example.

## Example 1:

### Input: 

**Activity**
| machine_id | process_id | activity_type | timestamp |
|------------|------------|---------------|-----------|
| 0          | 0          | start         | 0.712     |
| 0          | 0          | end           | 1.520     |
| 0          | 1          | start         | 3.140     |
| 0          | 1          | end           | 4.120     |
| 1          | 0          | start         | 0.550     |
| 1          | 0          | end           | 1.550     |
| 1          | 1          | start         | 0.430     |
| 1          | 1          | end           | 1.420     |
| 2          | 0          | start         | 4.100     |
| 2          | 0          | end           | 4.512     |
| 2          | 1          | start         | 2.500     |
| 2          | 1          | end           | 5.000     |

### Output: 

| machine_id | processing_time |
|------------|-----------------|
| 0          | 0.894           |
| 1          | 0.995           |
| 2          | 1.456           |

# Intuition
<!-- Describe your first thoughts on how to solve this problem. -->
The key is to calculate the duration of each process by matching 'start' and 'end' events for each machine, then average these durations. We need a way to link these events, considering they represent different states of the same process.

# Approach
<!-- Describe your approach to solving the problem. -->
- **Solution 1:** Use a Common Table Expression (CTE) to first match 'start' with 'end' events, compute durations, and then average them.
- **Solution 2:** Use a direct join to match 'start' and 'end' events, compute and average durations in one step.

# Complexity
- **Time complexity**: 
  - Both Solutions: O(n log n) in the worst case where n is the number of records due to join operations. With indexing, this can approach O(n).

- **Space complexity**: 
  - **Solution 1:** O(n) for CTE storage, then O(k) for the result where k is the number of machines.
  - **Solution 2:** O(n) for join operations, then O(k) for the result.

# Solution 1 (Using CTE)

```mysql
WITH start_end_times AS (
    SELECT 
        a1.machine_id,
        a2.timestamp - a1.timestamp AS process_duration
    FROM 
        Activity a1
    JOIN 
        Activity a2 ON a1.machine_id = a2.machine_id 
                   AND a1.process_id = a2.process_id 
                   AND a1.activity_type = 'start' 
                   AND a2.activity_type = 'end'
)
SELECT 
    machine_id,
    ROUND(AVG(process_duration), 3) AS processing_time
FROM 
    start_end_times
GROUP BY 
    machine_id;
```

# Solution 2 (Direct Join)
``` mysql
SELECT 
    a1.machine_id,
    ROUND(AVG(a2.timestamp - a1.timestamp), 3) AS processing_time
FROM 
    Activity a1
JOIN 
    Activity a2 ON a1.machine_id = a2.machine_id 
               AND a1.process_id = a2.process_id 
               AND a1.activity_type = 'start' 
               AND a2.activity_type = 'end'
GROUP BY 
    a1.machine_id;